# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

### Milvus

In [1]:
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )

C:\Users\KeychronK3\AppData\Local\Temp\ipykernel_14944\3633838239.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")
C:\Users\KeychronK3\AppData\Local\Temp\ipykernel_14944\3633838239.py:6: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectordb = Milvus(


In [2]:
results = vectordb.similarity_search(query="VMAX", k=2)
# results

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [3]:
from langchain_community.llms import Ollama

my_llm = Ollama(base_url='http://localhost:11434', model='deepseek-r1:1.5b', temperature=0.1)

# my_llm.invoke("你好")

C:\Users\KeychronK3\AppData\Local\Temp\ipykernel_14944\3226524278.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  my_llm = Ollama(base_url='http://localhost:11434', model='deepseek-r1:1.5b', temperature=0.1)


## 3. 构建检索问答链

prompts

In [4]:
from langchain.prompts import PromptTemplate

template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [5]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(my_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


## 4.检索问答链效果测试

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [6]:
questions = [
    "什么是VMAX？",
    "有哪些功能", 
    # "整理成表格"  
]

for question in questions:
    result = qa_chain({"query": question})  # Pass string directly, not dict
    # print(f"问题：{question}")
    print(f"回答：{result['result']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

/tmp/ipykernel_907333/2682531927.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})  # Pass string directly, not dict


回答：<think>
嗯，用户问的是“什么是VMAX？”首先，我需要明确VMAX是什么。根据提供的资料，VMAX是一个基于价值工程的网络评估工具，用于分析和优化网络性能。

接下来，我要解释VMAX的基本概念，包括它的核心功能、应用场景以及如何应用它来提升网络质量。同时，还要提到VMAX的优势，比如快速迭代和持续改进的特点。

然后，我需要考虑用户可能的背景知识，确保解释清晰易懂，避免过于专业的术语，或者如果有相关技术背景的话，可以稍微深入一些。最后，总结一下VMAX的价值，帮助用户理解它在运营商中的重要性。
</think>

VMAX（Value Stream Management）是一种基于价值工程和网络评估的技术工具，用于分析、优化和提升网络性能。它通过量化网络资源的使用效率和潜在收益，帮助运营商识别关键网络节点和路径，并制定相应的开发和维护策略。

### VMAX 的核心概念
1. **价值工程**：VMAX的核心思想是“价值工程”，即通过分析现有网络资源的使用效率，确定哪些资源可以被优化以提高整体网络性能。
2. **网络评估**：VMAX通过定期或周期性评估网络性能指标（如带宽、延迟、响应时间等），识别出网络中的瓶颈和不足之处。
3. **资源分配**：VMAX帮助运营商合理分配网络资源，确保关键节点的高性能，并减少对非关键节点的负担。

### VMAX 的应用场景
1. **网络优化**：通过分析网络性能指标，VMAX可以帮助运营商识别并修复网络问题，提升整体网络稳定性。
2. **网络扩展**：VMAX支持多度（天、周、月粒度）评估，帮助运营商在不同时间或时间段内优化网络资源分配。
3. **网络建设**：VMAX可以用于确定网络发展策略，如增加带宽、升级核心网等，以满足业务需求。

### VMAX 的优势
1. **快速迭代**：VMAX支持持续改进的特性，能够快速识别和修复网络问题，提升运营效率。
2. **资源优化**：通过量化评估，VMAX帮助运营商合理分配网络资源，避免对非关键节点造成负担。
3. **价值导向**：VMAX以用户需求为导向，确保网络优化始终围绕提高业务质量和服务体验。

### 总结
VMAX是一种高效、灵活的网络评估工具，能够帮助运营商快速识别和优化网络性能，提升整体网络运营效率。它通过量化评估和持续改进的特点，

In [7]:
question_1 = "什么是vmax的上网日志系统？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
<think>
嗯，我现在要弄清楚VMAX的上网日志系统是什么。首先，我看到用户的问题是“什么是VMAX的上网日志系统？”，看起来他们想了解这个系统的定义和功能。

从提供的文档来看，VMAX是一个网络管理工具，主要支持云化和分片化的网络架构。在文档中提到，VMAX可以保存用户的上网日志，并通过不同的接口进行查询和分析。比如，用户可以通过Windows应用程序、浏览器等方式访问界面，或者使用RESTful接口来访问服务器端的服务。

另外，文档里提到了NAT日志的入库功能，支持将NAT日志保存到GP数据库，还可以通过云化核心网、4G/5G控制面、软采XDR查询、VOLTEXDR查询和原始码流查询。还有云化上网日志XDR查询功能，包括核心网、软采XDR、VoLTEXDR和原始码流。

还有NAT日志的历史查询功能，当有两个库分别存储2个月和6个月的数据时，可以使用历史数据查询6个月的数据。另外，VMAX还支持云化网关对接，包括用户访问日志、上下线日志、NAT日志上传到SFTP，以及用户监控接口等。

拨测结果自动比对功能也是VMAX的一个重要部分，它将拨测记录文件上传到指定目录，然后定时扫描FTP目录下拨测文件，提取数据进行比对，并存入GP数据库。这些功能说明了VMAX在云化网络管理中的应用和扩展能力。

综合来看，VMAX的上网日志系统是一个强大的网络管理系统，能够支持用户日志的保存、查询、分析和管理，同时还能与其他云化网络架构无缝对接，满足分片化的需求。
</think>

VMAX（Virtual Application Management）的上网日志系统是一种用于存储和管理用户网络访问日志的网络工具。它通过多种接口和功能模块，支持用户对网络访问记录的查询、分析和管理，并与云化网络架构进行集成，以实现分片化的网络管理需求。

### VMAX 的上网日志系统概述：
1. **功能模块**：
   - **采集层**：包括探针（Probe）、防火墙日志处理单元（FirewalllogProcess）和数据合成单元（DataMergeProcess），用于采集和抓包网络信令。
   - **存储共享层**：采用分布式存储集群，支持MPP数据库（Gbase）和本地分布式文件系统（HDFS），以保存用户上网日志。


In [8]:
question_2 = "严威是谁？"
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
<think>
嗯，用户的问题是“严威是谁？”。看起来这是一个关于VMAX-S系统配置或日志分析的查询。首先，我需要理解用户的具体需求是什么。

用户可能是在进行系统调试或者优化，遇到了某个问题，想找出严威的身份。严威可能是某个服务、节点或者参数的名字，或者是某个特定的配置选项。为了更好地帮助用户，我应该查看提供的日志和图1-2，看看有没有提到严威相关的信息。

在提供的日志中，有很多关于进程使用率、区域节点数、GC花费时间等的信息，但没有看到任何与“严威”相关的内容。此外，在图1-2中，端到端系统功能部分也没有提到严威这个名字。

可能的情况是，用户是在配置VMAX-S时遇到了问题，或者在分析日志时发现某个节点或服务的参数被误识别为严威。也有可能是用户对系统的某些高级配置选项感到困惑，需要进一步了解。

考虑到这些信息，我应该建议用户提供更多的细节，比如查看特定的日志记录、系统配置文件或其他相关资料，以便更准确地帮助他们定位严威的身份。同时，如果用户已经知道严威的名字，我可以提供一些关于如何使用VMAX-S进行配置和优化的指导，帮助他们解决问题。

总的来说，用户需要的是关于“严威”在VMAX-S中的具体信息，但目前提供的数据中没有相关线索。因此，我建议用户提供更多的背景信息或日志记录以便更有效地解答问题。
</think>

根据您提供的日志和图1-2，似乎没有提到“严威”的相关信息。如果您能提供更多细节或查看特定的日志记录，我可以更好地帮助您定位严威的身份或提供相关的配置指导。

如果“严威”是某个节点、服务或参数的名字，请结合日志和其他信息进一步分析。


### 4.2 无知识库大模型自己回答的效果

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
my_llm.predict(prompt_template)

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
my_llm.predict(prompt_template)

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [6]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

C:\Users\will\AppData\Local\Temp\ipykernel_23612\2228008247.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
# 修改后的Prompt模板（添加chat_history变量）
template = """你是VMAX运维助手，请参考以下对话历史和上下文来回答问题：
    {chat_history}
    
    相关上下文：
    {context}
    
    问题：{question}
    回答结束时说“谢谢你的提问！”
    """
    
QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=template
    )
    
# 创建对话链
qa_chain = ConversationalRetrievalChain.from_llm(
        llm=my_llm,
        retriever=vectordb.as_retriever(),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": QA_PROMPT},
        chain_type="stuff"
    )
    
    # result = qa_chain({"question": question})
    # return result["answer"]
    

然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [ ]:
questions = [
    "什么是VMAX？",
    "有哪些功能", 
    # "整理成表格"  
]

for question in questions:
    result = qa_chain({"question": question})  # Pass string directly, not dict
    # print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

回答：VMAX是ValueMAX端到端多维价值分析系统的简称，面向用户的网络运维和运营分析产品。它立足于从客户角度去感知和分析网络和业务信息，并通过实时的分析、事后分析等方式为运营商提供全面支撑。


